In [ ]:
%%capture
%run setup.py build_ext --inplace

In [ ]:
import sys
from itertools import product, starmap, tee
from pprint import pprint
from pickle import dump, load

import sage.all
from klrw.perfect_complex import (KLRWHomOfPerfectComplexes, )
from klrw.combinatorial_ebranes import CombinatorialEBrane
folder_path = "./pickles/"

In [ ]:
def get_base(E):
    cls, args, kwrds = E.klrw_algebra[k]._reduction
    R = QQ
    kwrds["base_R"] = R
    kwrds['vertex_scaling'] = False
    kwrds['edge_scaling'] = False
    kwrds["default_vertex_parameter"] = R.one()
    kwrds["default_edge_parameter"] = R.one()
    return cls(*args, **kwrds)

In [ ]:
unlink_name = "0_1^2"
unknot_name = "0_1"

## Maps from the unlink to itself

In [ ]:
n, k = 4, 2

E = CombinatorialEBrane(n, k)
# no braid to apply

In [ ]:
E_complex = E.complex(
    *range(k),
    pickle="save",
    link_name=unlink_name,
    folder_path=folder_path,
).base_change(get_base(E))

**Collect the dimensions of the ext spaces into a dict**

In [ ]:
%%time
MAX_DEG = 30

# TQDM seems to not work with sage noteboks :(
#for (i, shift), ext in tqdm(zip(degrees, exts):

exts = {}
ext_dims = {}
total = (2 * MAX_DEG)**2
progress = 0
for shift, i in product(range(-MAX_DEG, MAX_DEG), range(-MAX_DEG, MAX_DEG)):
    rhom = KLRWHomOfPerfectComplexes(E_complex, E_complex[shift,i])
    ext = rhom.ext(0)
    dim = ext.dimension()
    if dim:
        exts[shift, i] = ext
        ext_dims[shift, i] = dim
    
    progress += 1
    sys.stdout.write(f"\rCompleted {progress}/{total} exts, now computing ({shift},{i})")

In [ ]:
pprint(ext_dims)

In [ ]:
print(len(exts))

In [ ]:
with open(folder_path + link_name + "_exts.pickle", "wb") as f:
    dump(exts, file=f)

## Cobordism(s) from the unknot to the unlink

In [ ]:
n, k = 4, 2

E_unlink = CombinatorialEBrane(n, k)
# no braid to apply

E_unknot = CombinatorialEBrane(n, k)
E_unknot.apply_braid([2])

In [ ]:
E_unlink.branes

In [ ]:
E_unknot.branes

In [ ]:
E_unlink_complex = E_unlink.complex(
    *range(k),
    #pickle="save",
    #link_name=unlink_name,
    #folder_path=folder_path,
).base_change(get_base(E_unlink))

E_unknot_complex = E_unknot.complex(
    *range(k),
    #pickle="save",
    #link_name=unknot_name,
    #folder_path=folder_path,
).base_change(get_base(E_unknot))

In [ ]:
rhom = KLRWHomOfPerfectComplexes(E_unknot_complex, E_unlink_complex)

In [ ]:
# There are maps from shift = -4 to shift = 1, mainly id
rhom[-4].basis().values()

In [ ]:
rhom.ext(0) ## now works!